In [1]:
# Import common libraries

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
from dnn_INSE_6180 import DNN
from dagmm_INSE_6180 import DAGMM
from sae_INSE_6180 import SAE

# Experiment 1

In [2]:
load_path = './datasets/chiller/df_dataset/'

def load_data(load_path, filename):
    csv_load_path = os.path.join(load_path, filename)
    return pd.read_csv(csv_load_path)

data = load_data(load_path, 'chiller10.csv') # Chiller data of Severity Level 1
data = data.drop('Unnamed: 0', axis='columns')
data_trn, data_tst = train_test_split(data, test_size=0.5, random_state=0)
X_trn, y_trn = data_trn.iloc[:, :-1], data_trn.iloc[:, -1]
X_tst, y_tst = data_tst.iloc[:, :-1], data_tst.iloc[:, -1]

# Scale data
scaler = StandardScaler().fit(X_trn)
X_trn_scaled, X_tst_scaled = scaler.transform(X_trn), scaler.transform(X_tst)

In [3]:
## Dataset 1 - 10% of Origninal data (2076 labeled data samples)
data_trn_sub, _ = train_test_split(data_trn, test_size=0.9, random_state=0)
X_trn_d1, y_trn_d1 = data_trn_sub.iloc[:, :-1], data_trn_sub.iloc[:, -1]

# Scaling data
X_trn_scaled_d1 = scaler.transform(X_trn_d1)

In [4]:
## Dataset 2 - 3% of Origninal training data (622 labeled data samples)
data_trn_sub, _ = train_test_split(data_trn, test_size=0.97, random_state=0)
X_trn_d2, y_trn_d2 = data_trn_sub.iloc[:, :-1], data_trn_sub.iloc[:, -1]

# Scaling data
X_trn_scaled_d2 = scaler.transform(X_trn_d2)

In [5]:
## Dataset 3 - 1% of Origninal training data (207 labeled data samples)
data_trn_sub, _ = train_test_split(data_trn, test_size=0.99, random_state=0)
X_trn_d3, y_trn_d3 = data_trn_sub.iloc[:, :-1], data_trn_sub.iloc[:, -1]

# Scaling data
X_trn_scaled_d3 = scaler.transform(X_trn_d3)

In [6]:
## Dataset 4 - 0.5% of Origninal data (103 labeled data samples)
data_trn_sub, _ = train_test_split(data_trn, test_size=0.995, random_state=0)
X_trn_d4, y_trn_d4 = data_trn_sub.iloc[:, :-1], data_trn_sub.iloc[:, -1]

# Scaling data
X_trn_scaled_d4 = scaler.transform(X_trn_d4)

# Choose dataset

In [7]:
### Select labeled datasets with different sizes to test model

# # Dataset 1 - 10% of Original dataset
# X_trn_, X_tst_ = X_trn_d1, X_tst
# X_trn_scaled_, X_tst_scaled_ = X_trn_scaled_d1, X_tst_scaled
# y_trn_, y_tst_ = y_trn_d1, y_tst

# # Dataset 2 - 3% of Origninal data
# X_trn_, X_tst_ = X_trn_d2, X_tst
# X_trn_scaled_, X_tst_scaled_ = X_trn_scaled_d2, X_tst_scaled
# y_trn_, y_tst_ = y_trn_d2, y_tst

## Dataset 3 - 1% of Origninal data
X_trn_, X_tst_ = X_trn_d3, X_tst
X_trn_scaled_, X_tst_scaled_ = X_trn_scaled_d3, X_tst_scaled
y_trn_, y_tst_ = y_trn_d3, y_tst

# ## Dataset 4 - 0.5% of Origninal data
# X_trn_, X_tst_ = X_trn_d4, X_tst
# X_trn_scaled_, X_tst_scaled_ = X_trn_scaled_d4, X_tst_scaled
# y_trn_, y_tst_ = y_trn_d4, y_tst

X_trn_.shape

(207, 65)

# DNN classifier without pretraining

In [8]:
# Without pretrain

out_dir = './results/'

# Create and train DNN_classifier
DNN_classifier = DNN(dnn_hiddens=[100, 50, 20], output_size=8, out_directory=out_dir,
                     pretrained_model=None, normalize=True, rate=0, n_epochs=100, pretrain_sae=False,
                     pretrain_dagmm=False, monte_carlo=True)
DNN_classifier.build_model(X_trn_)
DNN_classifier.fit((X_trn_, y_trn_))

print("Training accuracry: %.2f%%" %(100*DNN_classifier.score((X_trn_, y_trn_))))
print("Test accuracry: %.2f%%" %(100*DNN_classifier.score((X_tst_, y_tst_))))

y_tst_pred_ = DNN_classifier.predict(X_tst_)
print("====================================================================")
print("Confusion matrix: \n", confusion_matrix(y_tst_, y_tst_pred_))
print("====================================================================")
print("Classification report: \n", classification_report(y_tst_, y_tst_pred_, digits=4))
print("====================================================================")

Epoch 1/100
6/6 [==============================] - 2s 214ms/step - loss: 2.7399 - accuracy: 0.1130 - val_loss: 2.0887 - val_accuracy: 0.2143
Epoch 2/100
6/6 [==============================] - 0s 11ms/step - loss: 2.1988 - accuracy: 0.2085 - val_loss: 1.8123 - val_accuracy: 0.3810
Epoch 3/100
6/6 [==============================] - 0s 10ms/step - loss: 1.8561 - accuracy: 0.3673 - val_loss: 1.6187 - val_accuracy: 0.5000
Epoch 4/100
6/6 [==============================] - 0s 10ms/step - loss: 1.5665 - accuracy: 0.5056 - val_loss: 1.4022 - val_accuracy: 0.6190
Epoch 5/100
6/6 [==============================] - 0s 10ms/step - loss: 1.3713 - accuracy: 0.5820 - val_loss: 1.2613 - val_accuracy: 0.7143
Epoch 6/100
6/6 [==============================] - 0s 10ms/step - loss: 1.1734 - accuracy: 0.7202 - val_loss: 1.1521 - val_accuracy: 0.7857
Epoch 7/100
6/6 [==============================] - 0s 10ms/step - loss: 1.1096 - accuracy: 0.7605 - val_loss: 1.0813 - val_accuracy: 0.8095
Epoch 8/100
6/6 [==

# DNN classifier using SAE as pretrained network

In [14]:
# Pretrain using SAE (Using SAE as pretrained model which is then used to pretrain DNN)

out_dir = './results/'
# create SAE model and train the model using unlabeled data
model_sae = SAE(sae_hiddens=[100, 50, 20], out_directory = out_dir, dropout_rate=0, n_epochs=100, normalize=True)
model_sae.build_model(X_trn) # X_trn is the unlabeled data ()
model_sae.fit(X_trn) # Fit and save the best SAE model to the out_dir path

Epoch 1/100
553/584 [===========================>..] - ETA: 0s - loss: 5.5689WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'tuple'> input: (<tf.Tensor 'IteratorGetNext:0' shape=(None, 65) dtype=float32>, <tf.Tensor 'IteratorGetNext:1' shape=(None, 65) dtype=float32>)
Consider rewriting this model with the Functional API.
584/584 [==============================] - 2s 2ms/step - loss: 5.4663 - val_loss: 1.1658
Epoch 2/100
584/584 [==============================] - 1s 1ms/step - loss: 1.0721 - val_loss: 0.3446
Epoch 3/100
584/584 [==============================] - 1s 1ms/step - loss: 0.5440 - val_loss: 0.2146
Epoch 4/100
584/584 [==============================] - 1s 1ms/step - loss: 0.4194 - val_loss: 0.1838
Epoch 5/100
584/584 [==============================] - 1s 1ms/step - loss: 0.3683 - val_loss: 0.1715
Epoch 6/100
584/584 [==============================] - 1s 1ms/step - loss: 0.4258 - val_loss: 0.1603
Epoch 7/100
584/5

In [15]:
# Using SAE to pretrain

sae_model = model_sae.restore() # load sae_mode saved in the previous step

# apply pretraining technique using sae_model as pretrained network
DNN_classifier = DNN(dnn_hiddens=[100, 50, 20], output_size=8, out_directory=out_dir,
                     pretrained_model=sae_model, normalize=True, rate=0, n_epochs=100, pretrain_sae=True, 
                     pretrain_dagmm=False, monte_carlo=True)
DNN_classifier.build_model(X_trn_)
DNN_classifier.fit((X_trn_, y_trn_))

print("Training accuracry: %.2f%%" %(100*DNN_classifier.score((X_trn_, y_trn_))))
print("Test accuracry: %.2f%%" %(100*DNN_classifier.score((X_tst_, y_tst_))))

y_tst_pred_ = DNN_classifier.predict(X_tst_)
print("====================================================================")
print("Confusion matrix: \n", confusion_matrix(y_tst_, y_tst_pred_))
print("====================================================================")
print("Classification report: \n", classification_report(y_tst_, y_tst_pred_, digits=4))
print("====================================================================")

Epoch 1/10
6/6 [==============================] - 1s 44ms/step - loss: 2.1318 - accuracy: 0.1290 - val_loss: 2.1968 - val_accuracy: 0.0714
Epoch 2/10
6/6 [==============================] - 0s 10ms/step - loss: 2.1488 - accuracy: 0.1440 - val_loss: 2.1851 - val_accuracy: 0.0714
Epoch 3/10
6/6 [==============================] - 0s 9ms/step - loss: 2.1357 - accuracy: 0.1195 - val_loss: 2.1737 - val_accuracy: 0.0952
Epoch 4/10
6/6 [==============================] - 0s 10ms/step - loss: 2.1303 - accuracy: 0.1251 - val_loss: 2.1595 - val_accuracy: 0.0952
Epoch 5/10
6/6 [==============================] - 0s 10ms/step - loss: 2.0953 - accuracy: 0.1485 - val_loss: 2.1486 - val_accuracy: 0.0952
Epoch 6/10
6/6 [==============================] - 0s 33ms/step - loss: 2.1128 - accuracy: 0.1317 - val_loss: 2.1385 - val_accuracy: 0.1190
Epoch 7/10
6/6 [==============================] - 0s 10ms/step - loss: 2.1049 - accuracy: 0.1463 - val_loss: 2.1281 - val_accuracy: 0.1190
Epoch 8/10
6/6 [============

# DNN classifier using DAGMM as pretrained network

In [11]:
# Pretrain using DAGMM
out_dir = './results/'
# create dagmm model and train the model using unlabeled data
model_dagmm = DAGMM(comp_hiddens=[100, 50, 20], comp_activation="elu",
                    est_hiddens=[10, 3], est_activation="elu", est_dropout_ratio=0.1,
                    n_epochs=30, batch_size=1024, normalize=True)
model_dagmm.build(X_trn)
model_dagmm.fit(X_trn)

Epoch 1/30
20764/20764 [==============================] - mean_loss: 66.2602 - val_loss: 22.3701
Best Epoch: 1
Epoch 2/30
20764/20764 [==============================] - mean_loss: 25.7874 - val_loss: 15.4399
Best Epoch: 2
Epoch 3/30
20764/20764 [==============================] - mean_loss: 22.6745 - val_loss: 11.9801
Best Epoch: 3
Epoch 4/30
20764/20764 [==============================] - mean_loss: 15.6502 - val_loss: 10.1659
Best Epoch: 4
Epoch 5/30
20764/20764 [==============================] - mean_loss: 16.6603 - val_loss: 6.6004
Best Epoch: 5
Epoch 6/30
20764/20764 [==============================] - mean_loss: 13.3023 - val_loss: 8.0818
Best Epoch: 5
Epoch 7/30
20764/20764 [==============================] - mean_loss: 13.1718 - val_loss: 4.6557
Best Epoch: 7
Epoch 8/30
20764/20764 [==============================] - mean_loss: 12.2453 - val_loss: 5.3291
Best Epoch: 7
Epoch 9/30
20764/20764 [==============================] - mean_loss: 12.5681 - val_loss: 4.6998
Best Epoch: 7
Epoch 

In [12]:
# Using DAGMM to pretrain DNN

dagmm_model = model_dagmm.restore() # load dagmm model saved in the previous step
out_dir = './results/'

# apply pretraining technique using dagmm as pretrained network
DNN_classifier = DNN(dnn_hiddens=[100, 50, 20], output_size=8, out_directory=out_dir,
                     pretrained_model=dagmm_model, normalize=True, rate=0, n_epochs=100, pretrain_sae=False,
                     pretrain_dagmm=True, monte_carlo=True)
DNN_classifier.build_model(X_trn_)
DNN_classifier.fit((X_trn_, y_trn_))

print("Training accuracry: %.2f%%" %(100*DNN_classifier.score((X_trn_, y_trn_))))
print("Test accuracry: %.2f%%" %(100*DNN_classifier.score((X_tst_, y_tst_))))

y_tst_pred_ = DNN_classifier.predict(X_tst_)
print("====================================================================")
print("Confusion matrix: \n", confusion_matrix(y_tst_, y_tst_pred_))
print("====================================================================")
print("Classification report: \n", classification_report(y_tst_, y_tst_pred_, digits=4))
print("====================================================================")

Epoch 1/10
6/6 [==============================] - 1s 45ms/step - loss: 2.2479 - accuracy: 0.1350 - val_loss: 2.2203 - val_accuracy: 0.0238
Epoch 2/10
6/6 [==============================] - 0s 11ms/step - loss: 2.2367 - accuracy: 0.1089 - val_loss: 2.2106 - val_accuracy: 0.0476
Epoch 3/10
6/6 [==============================] - 0s 12ms/step - loss: 2.2306 - accuracy: 0.1310 - val_loss: 2.2003 - val_accuracy: 0.0238
Epoch 4/10
6/6 [==============================] - 0s 12ms/step - loss: 2.1955 - accuracy: 0.1235 - val_loss: 2.1890 - val_accuracy: 0.0476
Epoch 5/10
6/6 [==============================] - 0s 12ms/step - loss: 2.2255 - accuracy: 0.1107 - val_loss: 2.1795 - val_accuracy: 0.0476
Epoch 6/10
6/6 [==============================] - 0s 12ms/step - loss: 2.2576 - accuracy: 0.0802 - val_loss: 2.1699 - val_accuracy: 0.0476
Epoch 7/10
6/6 [==============================] - 0s 12ms/step - loss: 2.1844 - accuracy: 0.1109 - val_loss: 2.1591 - val_accuracy: 0.0476
Epoch 8/10
6/6 [===========

6/6 [==============================] - 0s 11ms/step - loss: 0.1277 - accuracy: 0.9821 - val_loss: 0.2548 - val_accuracy: 0.9286
Epoch 50/100
6/6 [==============================] - 0s 11ms/step - loss: 0.1183 - accuracy: 0.9787 - val_loss: 0.2428 - val_accuracy: 0.9286
Epoch 51/100
6/6 [==============================] - 0s 12ms/step - loss: 0.1199 - accuracy: 0.9809 - val_loss: 0.2521 - val_accuracy: 0.9286
Epoch 52/100
6/6 [==============================] - 0s 12ms/step - loss: 0.1154 - accuracy: 0.9869 - val_loss: 0.2390 - val_accuracy: 0.9286
Epoch 53/100
6/6 [==============================] - 0s 11ms/step - loss: 0.1189 - accuracy: 0.9817 - val_loss: 0.2423 - val_accuracy: 0.9286
Epoch 54/100
6/6 [==============================] - 0s 33ms/step - loss: 0.1175 - accuracy: 0.9869 - val_loss: 0.2337 - val_accuracy: 0.9286
Epoch 55/100
6/6 [==============================] - 0s 12ms/step - loss: 0.1141 - accuracy: 0.9791 - val_loss: 0.2236 - val_accuracy: 0.9286
Epoch 56/100
6/6 [========

# Baseline models

In [16]:
## 1. KNN MODEL

print("KNN MODEL:")

k = [i for i in range(2,10)]
p = [j for j in range(1,3)]
param_grid = [{'n_neighbors': k, 'p': p}]
knn_grid_search = GridSearchCV(KNeighborsClassifier(), param_grid=param_grid, cv=3,
                           scoring='f1_weighted', n_jobs=-1, return_train_score=True,
                           verbose=1)

knn_grid_search.fit(X_trn_scaled_, y_trn_)
knn_clf = knn_grid_search.best_estimator_
print(knn_clf)

print("Train Accuracy:", 100*knn_clf.score(X_trn_scaled_, y_trn_), chr(37))
print("Test Accuracy:", 100*knn_clf.score(X_tst_scaled_, y_tst_), chr(37))

y_tst_pred_ = knn_clf.predict(X_tst_scaled_)
print("====================================================================")
print("Confusion matrix: \n", confusion_matrix(y_tst_, y_tst_pred_))
print("====================================================================")
print("Classification report: \n", classification_report(y_tst_, y_tst_pred_, digits=4))
print("====================================================================")

## 2. SVM MODEL

print("SVM MODEL:")

C = [2**i for i in range(0, 4)]
gamma = [2**j for j in range(-2,2)]
param_grid = [{'C': C, 'gamma': gamma}]
svm_grid_search = GridSearchCV(SVC(), param_grid=param_grid, cv=3,
                           scoring='f1_weighted', n_jobs=-1, return_train_score=True,
                           verbose=1)
svm_grid_search.fit(X_trn_scaled_, y_trn_)
svm_clf = svm_grid_search.best_estimator_
print(svm_clf)

print("Train Accuracy:", 100*svm_clf.score(X_trn_scaled_, y_trn_), chr(37))
print("Test Accuracy:", 100*svm_clf.score(X_tst_scaled_, y_tst_), chr(37))

y_tst_pred_ = svm_clf.predict(X_tst_scaled_)
print("====================================================================")
print("Confusion matrix: \n", confusion_matrix(y_tst_, y_tst_pred_))
print("====================================================================")
print("Classification report: \n", classification_report(y_tst_, y_tst_pred_, digits=4))
print("====================================================================")

## 3. RANDOM FOREST MODEL

print("RANDOM FOREST MODEL:")

n_estimators_ = [int(x) for x in np.linspace(100, 550, 10)]

param_grid = {'n_estimators':n_estimators_}
rf_grid_search = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, cv=3,
                              scoring='f1_weighted', n_jobs=-1, return_train_score=True,
                              verbose=1)
rf_grid_search.fit(X_trn_, y_trn_)

rf_clf = rf_grid_search.best_estimator_

print(rf_clf)

print("Train Accuracy:", 100*rf_clf.score(X_trn_, y_trn_), chr(37))
print("Test Accuracy:", 100*rf_clf.score(X_tst_, y_tst_), chr(37))

y_tst_pred_ = rf_clf.predict(X_tst_)
print("====================================================================")
print("Confusion matrix: \n", confusion_matrix(y_tst_, y_tst_pred_))
print("====================================================================")
print("Classification report: \n", classification_report(y_tst_, y_tst_pred_, digits=4))
print("====================================================================")



# ## 4. LOGISTIC REGRESSION MODEL

# C= np.logspace(-4,4,9)
# solver=['newton-cg']

# param_grid = [{'solver': solver, 'C': C}]
# lr_grid_search = GridSearchCV(LogisticRegression(max_iter=100),
#                            param_grid=param_grid, cv=3, scoring='f1_weighted',
#                            n_jobs=-1, return_train_score=True, verbose=1)
# lr_grid_search.fit(X_trn_scaled_, y_trn_)
# lr_clf = lr_grid_search.best_estimator_
# print(lr_clf)

# print("Train Accuracy:", 100*lr_clf.score(X_trn_scaled_, y_trn_), chr(37))
# print("Test Accuracy:", 100*lr_clf.score(X_tst_scaled_, y_tst_), chr(37))

## 5. ADABOOST MODEL

print("ADABOOST MODEL:")

param_grid = {
    'n_estimators' : [100, 200, 300],
    'learning_rate' : [0.001, 0.01, 0.1]
}
ad_grid_search = GridSearchCV(AdaBoostClassifier(), param_grid = param_grid,
                                cv=3, scoring='f1_weighted', n_jobs=-1, return_train_score=True, verbose=1
                                )
ad_grid_search.fit(X_trn_, y_trn_)

ad_clf = ad_grid_search.best_estimator_

print(ad_clf)

print("Train Accuracy:", 100*ad_clf.score(X_trn_, y_trn_), chr(37))
print("Test Accuracy:", 100*ad_clf.score(X_tst_, y_tst_), chr(37))

y_tst_pred_ = ad_clf.predict(X_tst_)
print("====================================================================")
print("Confusion matrix: \n", confusion_matrix(y_tst_, y_tst_pred_))
print("====================================================================")
print("Classification report: \n", classification_report(y_tst_, y_tst_pred_, digits=4))
print("====================================================================")


# ## 6. GAUSIAN NAIVE BAYSESSIAN

# params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
# gnb_grid_search = GridSearchCV(GaussianNB(), param_grid=params_NB, cv=3, scoring='accuracy',return_train_score=True)
# gnb_grid_search.fit(X_trn_scaled_, y_trn_)

# gnb_clf = gnb_grid_search.best_estimator_

# print(gnb_clf)

# print("Train Accuracy:", 100*gnb_clf.score(X_trn_scaled_, y_trn_), chr(37))
# print("Test Accuracy:", 100*gnb_clf.score(X_tst_scaled_, y_tst_), chr(37))


KNN MODEL:
Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:    2.2s finished


KNeighborsClassifier(n_neighbors=7)
Train Accuracy: 53.62318840579711 %
Test Accuracy: 25.317857830861108 %
Confusion matrix: 
 [[ 770  255  199  318  579  208  241    8]
 [ 779  443   37  241  618  196  264    8]
 [ 443  153 1176  158  394  239   13    0]
 [ 792  270  162  428  453  201  229   26]
 [ 825  271  130  425  512  202  205   84]
 [ 870  273  160  362  550  252  108   15]
 [ 513  172  131  194  352   81 1147   55]
 [ 582  116   82  166  654   98  347  529]]
Classification report: 
               precision    recall  f1-score   support

           0     0.1381    0.2987    0.1889      2578
           1     0.2268    0.1713    0.1952      2586
           2     0.5662    0.4565    0.5055      2576
           3     0.1867    0.1671    0.1764      2561
           4     0.1245    0.1929    0.1513      2654
           5     0.1706    0.0973    0.1239      2590
           6     0.4491    0.4336    0.4412      2645
           7     0.7297    0.2055    0.3207      2574

    accuracy  

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:    0.0s finished


Test Accuracy: 45.26584473126565 %
Confusion matrix: 
 [[ 971    1    0  418  392  207  501   88]
 [ 491  678    0   13  399   61  926   18]
 [   0    0 1237    0    0    0 1339    0]
 [ 427    0    0  717  732  203  348  134]
 [ 330    0    0  240 1283  157  400  244]
 [ 335    0    0  381  375  965  390  144]
 [  92    0    0   12  168    0 2134  239]
 [  48    0    0    0  516    0  596 1414]]
Classification report: 
               precision    recall  f1-score   support

           0     0.3604    0.3766    0.3684      2578
           1     0.9985    0.2622    0.4153      2586
           2     1.0000    0.4802    0.6488      2576
           3     0.4026    0.2800    0.3303      2561
           4     0.3320    0.4834    0.3936      2654
           5     0.6058    0.3726    0.4614      2590
           6     0.3217    0.8068    0.4600      2645
           7     0.6199    0.5493    0.5825      2574

    accuracy                         0.4527     20764
   macro avg     0.5801    0.4514

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    1.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    1.6s finished


RandomForestClassifier(n_estimators=250)
Train Accuracy: 100.0 %
Test Accuracy: 79.47408977075709 %
Confusion matrix: 
 [[1549    0   33  673   25  285   13    0]
 [   0 2579    7    0    0    0    0    0]
 [   0    0 2576    0    0    0    0    0]
 [ 170    0    8 1304  529  539   11    0]
 [  45    0   12  376 2067  144    8    2]
 [ 344    0   13  436  422 1373    1    1]
 [  20    0   13   33    0    0 2576    3]
 [   0    0   10   25   57    0    4 2478]]
Classification report: 
               precision    recall  f1-score   support

           0     0.7279    0.6009    0.6583      2578
           1     1.0000    0.9973    0.9986      2586
           2     0.9641    1.0000    0.9817      2576
           3     0.4580    0.5092    0.4822      2561
           4     0.6668    0.7788    0.7185      2654
           5     0.5865    0.5301    0.5569      2590
           6     0.9858    0.9739    0.9798      2645
           7     0.9976    0.9627    0.9798      2574

    accuracy          

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  27 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    1.1s finished


AdaBoostClassifier(learning_rate=0.01, n_estimators=100)
Train Accuracy: 79.22705314009661 %
Test Accuracy: 72.30784049316124 %
Confusion matrix: 
 [[ 951    0   10  269  115 1191   42    0]
 [   0 2578    7    0    1    0    0    0]
 [   0    0 2576    0    0    0    0    0]
 [ 148    0    7  328  898 1168   11    1]
 [   0    0   12  154 2463   14   10    1]
 [  30    0   13  225 1174 1147    1    0]
 [   4    0   13   14   26    2 2570   16]
 [   0    0   10    4  116   10   33 2401]]
Classification report: 
               precision    recall  f1-score   support

           0     0.8394    0.3689    0.5125      2578
           1     1.0000    0.9969    0.9985      2586
           2     0.9728    1.0000    0.9862      2576
           3     0.3300    0.1281    0.1845      2561
           4     0.5139    0.9280    0.6615      2654
           5     0.3247    0.4429    0.3747      2590
           6     0.9636    0.9716    0.9676      2645
           7     0.9926    0.9328    0.9617      